In [ ]:

from PIL import Image
import requests
import matplotlib.pyplot as plt
import torch.nn as nn
from torchinfo import summary
import torch
import pandas as pd
import numpy as np
import torch.optim as optim
import torchvision.transforms as T
from glob import glob
import segmentation_models_pytorch as smp
import torch.nn.functional as F
from collections import defaultdict
import cv2
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")
tf=T.ToTensor()


In [ ]:
params={'image_size':512,
        'lr':2e-4,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':1,
        'epochs':500,}

In [ ]:
image_path='../../data/external/ori/*.png'
mask1_path='../../data/external/mask/class1/*.png'

In [87]:
class CustomDataset(Dataset):
    def __init__(self, image_list, label_list):
        self.img_path = image_list
        self.label = label_list

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        image_path = Image.open(self.img_path[idx])
        image_path=tf(image_path)
        file_path=os.path.basename(self.img_path[idx])
        label1 = np.array(Image.open(self.label[idx]))
        label1=label1[:,:,0,np.newaxis]
        label2=np.array(Image.open(self.label[idx].replace('/class1', '/class2')))
        label2=label2[:,:,0,np.newaxis]
        label3=np.array(Image.open(self.label[idx].replace('/class1', '/class3')))
        label3=label3[:,:,0,np.newaxis]
        label4=np.array(Image.open(self.label[idx].replace('/class1', '/class4')))
        label4=label4[:,:,0,np.newaxis]
        label=np.concatenate((label1,label2,label3,label4),axis=2)
        label_path = tf(cv2.resize(label, (512, 512)))
       
        return image_path, label_path,file_path

test_dataset = CustomDataset(glob(image_path), glob(mask1_path))
test_dataloader = DataLoader(
    test_dataset, batch_size=params['batch_size'],shuffle=False, drop_last=True)


In [88]:
def dice_loss(pred, target, num_classes=4):
    smooth = 0
    dice_per_class = torch.zeros(num_classes).to(pred.device)

    for class_id in range(num_classes):
        pred_class = pred[:, class_id, ...]
        target_class = target[:, class_id, ...]

        intersection = torch.sum(pred_class * target_class)
        A_sum = torch.sum(pred_class * pred_class)
        B_sum = torch.sum(target_class * target_class)

        dice_per_class[class_id] =(2. * intersection + smooth) / (A_sum + B_sum + smooth)

    return dice_per_class

def compute_iou(pred_mask, true_mask, threshold=0.5, num_classes=4):
    """
    IoU를 계산하는 함수

    :param pred_mask: 모델이 예측한 마스크 (torch.Tensor)
    :param true_mask: 실제 마스크 (torch.Tensor)
    :param threshold: 이진화를 위한 임계값
    :return: IoU 값
    """
    iou_per_class = torch.zeros(num_classes).to(device)
    for class_id in range(num_classes):
    # 예측된 마스크 이진화
        pred_mask = (pred_mask[:,class_id, ...] > threshold).float()
        
        # 실제 마스크 이진화
        true_mask = (true_mask[:,class_id, ...] > threshold).float()
        
        # 교차 계산
        intersection = torch.sum(pred_mask * true_mask)
        
        # 합집합 계산
        union = torch.sum(pred_mask) + torch.sum(true_mask) - intersection
        
        # IoU 계산
        iou_per_class[class_id]= intersection / union
    
    return iou_per_class

def compute_f1(pred_mask, true_mask, threshold=0.5, num_classes=4):
    """
    F1 점수를 계산하는 함수

    :param pred_mask: 모델이 예측한 마스크 (torch.Tensor)
    :param true_mask: 실제 마스크 (torch.Tensor)
    :param threshold: 이진화를 위한 임계값
    :param num_classes: 클래스의 수
    :param device: 연산에 사용할 디바이스 (기본값: 'cpu')
    :return: 각 클래스별 F1 점수 (torch.Tensor)
    """
    f1_per_class = torch.zeros(num_classes).to(device)
    precision=torch.zeros(num_classes).to(device)
    recall=torch.zeros(num_classes).to(device)
    
    for class_id in range(num_classes):
        # 예측된 마스크 이진화
        pred_binary_mask = (pred_mask[:, class_id, ...] > threshold).float()
        
        # 실제 마스크 이진화
        true_binary_mask = (true_mask[:, class_id, ...] > threshold).float()
        
        # True Positive (TP), False Positive (FP), False Negative (FN) 계산
        TP = torch.sum(pred_binary_mask * true_binary_mask)
        FP = torch.sum(pred_binary_mask * (1 - true_binary_mask))
        FN = torch.sum((1 - pred_binary_mask) * true_binary_mask)
        
        # 정밀도 (Precision) 계산
        precision = TP / (TP + FP + 1e-8)  # 분모가 0이 되는 것을 방지하기 위해 작은 값 추가
        
        # 재현율 (Recall) 계산
        recall = TP / (TP + FN + 1e-8)  # 분모가 0이 되는 것을 방지하기 위해 작은 값 추가
        
        # F1 점수 계산
        f1_per_class[class_id] = 2 * (precision * recall) / (precision + recall + 1e-8)  # 분모가 0이 되는 것을 방지하기 위해 작은 값 추가
        precision[class_id]=precision
        recall[class_id]=recall
    
    return f1_per_class,precision,recall

model = smp.MAnet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=4,                      # model output channels (number of classes in your dataset)
).to(device)
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), lr=params['lr'], betas=(params['beta1'], params['beta2']))

In [89]:
for i in range(3):
    model.load_state_dict(torch.load('../../model/MANet/MANet_'+str(i+1)+'_check.pth'))
    df=pd.DataFrame(columns=['file_name','Dice1','Dice2','Dice3','Dice4','mDice','IoU1','IoU2','IoU3','IoU4','mIoU','f1','precision','recall'])
    with torch.no_grad():
        test = tqdm(test_dataloader)
        count = 0
        val_running_loss = 0.0
        acc_loss = 0
        for x, y,file_path in test:
            model.eval()
            y = y.to(device).float()
            count += 1
            x = x.to(device).float()
            predict = model(x).to(device)
            cost = dice_loss(predict, y)  # cost 구함
            iou=compute_iou(predict, y)
            f1,precision,recall=compute_f1(predict, y)
            
            df.loc[len(df)]=[file_path,cost[0].item(),cost[1].item(),cost[2].item(),cost[3].item(),cost.mean().item(),iou[0].item(),iou[1].item(),iou[2].item(),iou[3].item(),iou.mean().item(),f1.mean().item(),precision.mean().item(),recall.mean().item()]
            
            test.set_description(
                f"val_Step: {count+1}")
    df.to_csv('../../data/external/result/MANet/MANet_'+str(i+1)+'_result.csv',index=False)

  0%|          | 0/4106 [00:00<?, ?it/s]


IndexError: too many indices for tensor of dimension 1

In [98]:
(predict[:,1,...]> 0.5).float()

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:6')

In [56]:
import torch

def compute_f1(pred_mask, true_mask, threshold=0.5, num_classes=4, device='cpu'):
    """
    F1 점수를 계산하는 함수

    :param pred_mask: 모델이 예측한 마스크 (torch.Tensor)
    :param true_mask: 실제 마스크 (torch.Tensor)
    :param threshold: 이진화를 위한 임계값
    :param num_classes: 클래스의 수
    :param device: 연산에 사용할 디바이스 (기본값: 'cpu')
    :return: 각 클래스별 F1 점수 (torch.Tensor)
    """
    f1_per_class = torch.zeros(num_classes).to(device)
    
    for class_id in range(num_classes):
        # 예측된 마스크 이진화
        pred_binary_mask = (pred_mask[:, class_id, ...] > threshold).float()
        
        # 실제 마스크 이진화
        true_binary_mask = (true_mask[:, class_id, ...] > threshold).float()
        
        # True Positive (TP), False Positive (FP), False Negative (FN) 계산
        TP = torch.sum(pred_binary_mask * true_binary_mask)
        FP = torch.sum(pred_binary_mask * (1 - true_binary_mask))
        FN = torch.sum((1 - pred_binary_mask) * true_binary_mask)
        
        # 정밀도 (Precision) 계산
        precision = TP / (TP + FP + 1e-8)  # 분모가 0이 되는 것을 방지하기 위해 작은 값 추가
        
        # 재현율 (Recall) 계산
        recall = TP / (TP + FN + 1e-8)  # 분모가 0이 되는 것을 방지하기 위해 작은 값 추가
        
        # F1 점수 계산
        f1_per_class[class_id] = 2 * (precision * recall) / (precision + recall + 1e-8)  # 분모가 0이 되는 것을 방지하기 위해 작은 값 추가
    
    return f1_per_class

# 예시 사용
# 임의의 예측된 마스크와 실제 마스크 생성
pred_mask = torch.rand((1, 4, 256, 256)).to('cpu')  # 모델의 출력 예시
true_mask = torch.randint(0, 2, (1, 4, 256, 256)).float().to('cpu')  # 실제 마스크 예시

f1 = compute_f1(pred_mask, true_mask, threshold=0.5, num_classes=4, device='cpu')
print(f"F1 Score per class: {f1}")

F1 Score per class: tensor([0.4979, 0.5030, 0.5037, 0.4988])


In [ ]:
y

In [ ]:
y.s